In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

raw_weather_df = spark.table("weather.bronze.bronze_weather")
raw_weather_df.display()

In [0]:
json_stirng = raw_weather_df.select('raw_weather_data').first()[0]
weather_schema = schema_of_json(json_stirng)

In [0]:
prased_df = (
    raw_weather_df
    .withColumn("weather_data", from_json(col("raw_weather_data"), weather_schema))
)

prased_df.display()

In [0]:
silver_weather = (
    prased_df.select(
        "city",
        "ingestion_ts",
        col("weather_data.current_weather.time").alias("event_time"),
        col("weather_data.current_weather.temperature").alias("temperature"),
        col("weather_data.current_weather.windspeed").alias("windspeed"),
        "source"
    )
)

In [0]:
silver_weather.display()

In [0]:
silver_weather = (
    silver_weather.dropDuplicates(
        ["city", "event_time"]
    )
)

In [0]:
(
    silver_weather.write
    .format("delta")
    .mode("overwrite")
    .option("mergeSchema", "true")
    .save("/Volumes/weather/silver/data_files/weather/")
)